In [71]:
from pathlib import Path, PurePath
import glob
import os
import numpy as np
import csv
import pandas as pd
import time
import matplotlib.pyplot as plt

In [72]:
file_path = PurePath(os.getcwd())
dataset_path = file_path.parent.parent.parent.joinpath('eicu')

In [73]:
lab_file = pd.read_csv(dataset_path.joinpath('lab.csv'))
# lab_prev = lab_file['labname'].value_counts()
d_lab_file = pd.read_csv(dataset_path.joinpath('lab_delirium.csv'))
# lab_del_prev = lab_del_file['labname'].value_counts()

In [74]:
lab_file = lab_file[['patientunitstayid', 'labresultoffset', 'labname', 'labresultrevisedoffset']]
d_lab_file = d_lab_file[['patientunitstayid', 'labresultoffset', 'labname', 'labresultrevisedoffset']]

'labid', 'patientunitstayid', 'labresultoffset', 'labtypeid', 'labname', 'labresult','labresulttext', 'labmeasurenamesystem', 'labmeasurenameinterface', 'labresultrevisedoffset'

In [75]:
unique_ids = lab_file['patientunitstayid'].unique()
d_unique_ids = d_lab_file['patientunitstayid'].unique()
unique_labs = lab_file['labname'].unique()

In [76]:
print(unique_ids[0:10])
print(d_unique_ids[0:10])
print(unique_labs[0:10])

[141168 141178 141179 141194 141196 141197 141203 141208 141227 141229]
[149321 150934 151213 151708 152171 155126 156970 162054 163084 164338]
['fibrinogen' 'PT - INR' 'magnesium' 'PT' 'pH' 'urinary creatinine' 'MCH'
 'BUN' 'urinary sodium' 'HCO3']


In [77]:
columns = []
for lab in unique_labs:
    columns.append(lab + 'count')
    columns.append(lab + 'mean')
    columns.append(lab + 'median')
    columns.append(lab)

In [ ]:


for id in unique_ids:

    df_id = lab_file[lab_file['patientunitstayid'] == id]

    for lab in unique_labs:
        
        df_lab = df_id[df_id['labname'] == lab]
        # df_lab = df_lab.sort_values(axis=0, by='labresultoffset')
        # print(df_lab)
        times = np.sort(df_lab['labresultoffset'].to_numpy())
        rev_times = np.sort(df_lab['labresultrevisedoffset'].to_numpy())
        count = len(times)
        time_diff = times[1:] - times[:-1]
        time_diff = time_diff[time_diff != 0]
        rev_time_diff = rev_times[1:] - rev_times[:-1]
        rev_time_diff = rev_time_diff[rev_time_diff != 0]

        

In [86]:
df_lab = lab_file[lab_file['labname'] == 'fibrinogen']
# print(df_lab[df_lab['patientunitstayid'] == 2074269])
lab_count = df_lab['patientunitstayid'].value_counts()
print(lab_count.median())

1.0


In [99]:
def get_stats(arr : pd.DataFrame):
    if arr.empty:
        IQR = np.NaN
    else: 
        q3, q1 = np.percentile(arr, [75, 25])
        IQR = q3 - q1
    # mean = arr.mean()
    # median = arr.median()
    # var = arr.var()
    # minima = arr.min()
    # maxima = arr.max()
    # return mean, var, median, IQR, minima, maxima
    return arr.mean(), arr.var(), arr.median(), IQR, arr.min(), arr.max(), len(arr)

In [102]:


lab_count_arr = []
for lab in unique_labs:
    df_lab = lab_file[lab_file['labname'] == lab]
    lab_count = df_lab['patientunitstayid'].value_counts()
    mean, var, median, IQR, minima, maxima, count = get_stats(lab_count)
    lab_count_arr.append([lab, mean, var, median, IQR, minima, maxima, count])
count_df = pd.DataFrame(lab_count_arr, columns = ["lab", "mean", "var", "median", "IQR", "min", "max", "n"])



In [103]:
d_lab_count_arr = []
for lab in unique_labs:
    df_lab = d_lab_file[d_lab_file['labname'] == lab]
    lab_count = df_lab['patientunitstayid'].value_counts()
    mean, var, median, IQR, minima, maxima, count = get_stats(lab_count)
    d_lab_count_arr.append([lab, mean, var, median, IQR, minima, maxima, count])
d_count_df = pd.DataFrame(d_lab_count_arr, columns = ["lab", "mean", "var", "median", "IQR", "min", "max", "n"])


In [104]:
count_df.to_csv(file_path.joinpath('LabsPerPatientStats.csv'))
d_count_df.to_csv(file_path.joinpath('LabsPerPatient_delirium.csv'))